In [2]:
#Set-up & Widget
# Upload-Widget (ipywidgets 8.x) + PDF-Ingestion
import sys, pathlib, importlib, ipywidgets as widgets
from IPython.display import display

# ----- Pfade ---------------------------------------------------------------
PROJECT_ROOT = pathlib.Path.cwd().parents[1]          # …/vocdata
SCRIPTS_DIR  = PROJECT_ROOT / "scripts"
PDF_DIR      = PROJECT_ROOT / "uploads" / "pdf"
PDF_DIR.mkdir(parents=True, exist_ok=True)

if str(SCRIPTS_DIR) not in sys.path:
    sys.path.insert(0, str(SCRIPTS_DIR))              # scripts/ importierbar

pdf_ingest = importlib.import_module("pdf_upload.ingest")    # Modul laden

# ----- Widget --------------------------------------------------------------
uploader = widgets.FileUpload(accept=".pdf", multiple=True, description="Upload")
display(uploader)

def handle_upload(change):
    for f in change["new"]:                           # Tuple mit Datei-Dicts
        dest = PDF_DIR / pathlib.Path(f["name"]).name
        dest.write_bytes(f["content"])                # Datei speichern
    pdf_ingest.ingest()                               # DB-Ingestion
    uploader.value = ()                               # Widget leeren
    print("🚀 Upload & Ingestion fertig.")

uploader.observe(handle_upload, names="value")


FileUpload(value=(), accept='.pdf', description='Upload', multiple=True)

In [4]:
#Schnelltest

# Einmalig ausführen, um Import & Ordner zu verifizieren
import pathlib, sys, pprint
print("PROJECT_ROOT :", PROJECT_ROOT)
print("pdf_ingest   :", pdf_ingest.__file__)
print("PDF_DIR      :", PDF_DIR)
print("uploads/pdf  – enthaltene Dateien:")
for p in PDF_DIR.glob("*.pdf"):
    print(" -", p.name)



PROJECT_ROOT : c:\Users\claud\iCloudDrive\Dokumente\02_CLI\Studium\ZHAW\Masterarbeit\vocdata
pdf_ingest   : c:\Users\claud\iCloudDrive\Dokumente\02_CLI\Studium\ZHAW\Masterarbeit\vocdata\scripts\pdf_upload\ingest.py
PDF_DIR      : c:\Users\claud\iCloudDrive\Dokumente\02_CLI\Studium\ZHAW\Masterarbeit\vocdata\uploads\pdf
uploads/pdf  – enthaltene Dateien:
 - Berufliche Passagen im Lebenslauf.pdf
 - bildungsbericht_2023.pdf
 - ch_lva_eba_schmid_neumann_2016.pdf
 - ch_lva_jugendsicht_neuenschwander_etal_1998.pdf
 - ch_lva_leva_schmid_etal_2008.pdf
 - ch_norway_lva_schmid_2016.pdf
 - ch_studie_lva_leva_beruf_negrini_etal_2015.pdf
 - ehb_trendbericht_lva_kriesi_etal_2016.pdf
 - euope_2014_early_leaving.pdf
 - Maurer_2013_BB_und_Arbeitsmarkt.pdf
